In [ ]:
from teehr import Evaluation
from pathlib import Path
import shutil

In [ ]:
# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "real_study")
shutil.rmtree(TEST_STUDY_DIR, ignore_errors=True)
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# TEST_DATA = "/home/matt/repos/teehr/tests/data/two_locations/"
# TEST_DATA = "/home/sam/git/teehr/tests/data/two_locations/"
TEST_DATA = "/home/slandsteiner/repos/teehr/tests/data/two_locations/"
LOCATIONS = Path(TEST_DATA, "two_locations.parquet")
XWALKS = Path(TEST_DATA, "two_crosswalks.parquet")
LOCATION_ATTRS = Path(TEST_DATA, "two_location_attributes.parquet")

In [ ]:
# Create an Evaluation object
ev = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
ev.enable_logging()

In [ ]:
# Clone the template
ev.clone_template()

In [ ]:
# Load the location data (observations)
ev.locations.load_spatial(in_path=LOCATIONS)

In [ ]:
# Load the timeseries data and map over the fields and set constants
ev.locations.to_geopandas()

In [ ]:
ev.fetch.usgs_streamflow(
    start_date="2000-10-01",
    end_date="2012-09-30"
)

In [ ]:
ev.primary_timeseries.to_pandas()

In [ ]:
# Load the crosswalk data
ev.location_crosswalks.load_parquet(
    in_path=XWALKS
)

In [ ]:
ev.fetch.nwm_retrospective_points(
    nwm_version="nwm30",
    variable_name="streamflow",
    start_date="2000-10-01",
    end_date="2012-09-30"
)

In [ ]:
ev.secondary_timeseries.to_pandas()

In [ ]:
from teehr.models.tables import (
    Attribute,
)
import duckdb

In [ ]:
df = duckdb.query(
    f"SELECT distinct(attribute_name) FROM read_parquet('{LOCATION_ATTRS}');"
).to_df()
attrs_list = [Attribute(name=i,type="catagorical", description=i) for i in list(df.attribute_name)]
# attrs_list

In [ ]:
# Add some attributes
ev.attributes.add(attrs_list)

In [ ]:
ev.location_attributes.load_parquet(LOCATION_ATTRS, field_mapping={"attribute_value": "value"})

In [ ]:
# Create the joined timeseries
ev.joined_timeseries.create(execute_udf=True)

In [ ]:
ev.joined_timeseries.to_pandas()